В данной статье авторы описывают нейросетевую архитектуру TabNet ... (**TODO**)

### Введение

Во введении кратко рассматривается вопрос о том, почему решающие деревья (DT) часто оказываются лучше нейронных сетей (DNN) на табличных данных. Авторы делают ряд утверждений, но впрочем не приводят к ним пояснений или доказательств:

> DT-based approaches ... are representionally efficient for decision manifolds with approximately hyperplane boundaries which are common in tabular data.

Это утверждение не вполне понятно: чтобы реализовать зависимость в виде гиперплоскости $y = ax_1 + bx_2 + c$, нейронной сети достаточно всего трех параметров, тогда как решающее дерево потребует множества параметров, чтобы хоть как-то приблизить эту зависимость.

> Previously-proposed DNN architectures are not well-suited for tabular data: e.g. stacked convolutional layers or multi-layer perceptrons (MLPs) are *vastly overparametrized* – the lack of appropriate inductive bias often causes them to fail to find optimal solutions for tabular decision manifolds (Goodfellow, Bengio, and Courville
2016; Shavitt and Segal 2018; Xu et al. 2019).

Здесь я тоже не до конца понимаю авторов. Наличие чрезмерной параметризации само по себе еще не означает, что модель плохая: это еще зависит от конкретного вида модели и алгоритма обучения. Например, нейронные сети бесконечной ширины (см. [Kernel Methods for Deep Learning]($Kernel Methods for Deep Learning$)) вполне могут хорошо обобщаться, несмотря на бесконечное число параметров. Кроме того, недавно был обнаружен эффект Deep Double Descent ([Nakkiran et al., 2019]($Deep Double Descent: Where Bigger Models and More Data Hurt$)), в котором тоже показывается, что при чрезмерной параметризации модель может хорошо обобщаться.

Мотивация к изучению методов глубокого обучения на табличных данных у авторов следующая:

1. Методы глубокого обучения хорошо масштабируются при увеличении объема обучающих данных ([Hestness et al., 2017]($Deep Learning Scaling is Predictable, Empirically$)). Хотя, с другой стороны, это же может быть справедливо и для градиентного бустинга.
2. Глубокое обучение открывает возможности для обучения представлений (representation learning), обучения на неразмеченных данных и переноса обучения (типичным примером является файн-тюнинг предобученных нейронных сетей в CV и NLP).
3. Благодаря end-to-end обучению можно совмещать табличные данные с изображениями и текстом.
4. Согласно авторам, модели глубокого обучения можно обучать на потоковых данных (learning from streaming data), хотя это же может быть справедливо и для градиентного бустинга.
5. Согласно авторам, в глубоком обучении снижается необходимость в feature engineering, по сравнению с решающими деревьями (хотя мне это не до конца понятно, скорее верно обратное, ведь монотонные преобразования признаков играют большую роль с нейронных сетях и практически не играют роли в решающих деревьях).

### Связанные работы

Говоря о чрезмерной параметризации нейронных сетей, авторы ссылаются на работу *Regularization Learning Networks: Deep Learning for Tabular Datasets* ([Shavitt and Segal, 2018]($Regularization Learning Networks: Deep Learning for Tabular Datasets$)). Авторы этой статьи высказывают гипотезу о том, что преимущество решающих деревьев перед нейронными сетями на табличных данных связано с тем, что в табличных данных разные признаки имеют разную степень важности. Из этого каким-то образом авторы заключают, что нужно вводить для каждого веса свой регуляризующий коэффициент (например, для L2-регуляризации).

> We hypothesized that this potentially large variability in the relative importance of different input features may partly explain the lower performance of DNNs on such tabular datasets. One way to overcome this limitation could be to assign a different regularization coefficient to every weight, which might allow the network to accommodate the non-distributed representation and the variability in relative importance found in tabular datasets.

Авторы предлагают параллельно с обучением весов сети обучать коэффициенты регуляризации на каждом весе с помощью минимизации специальной дополнительной функции потерь, которая равна функции потерь предсказания (например, MSE или кроссэнтропии) на селдующем батче, из которой градиент распространяется в текущий батч. Таким образом, мы подталкиваем коэффициенты регуляризации к тому, чтобы они минимизировали функцию потерь предсказания на следующем батче.

Таким образом, в статье описана мысль о собственном регуляризующем коэффициенте для каждого веса сети. Это похоже на метод automatic relevance determination ([Neal, 1995]($Bayesian Learning for Neural Networks$), см. также [Schmidt, 2018](https://www.cs.ubc.ca/~schmidtm/Courses/540-W18/L32.pdf)), где аналогичный подход адаптивной регуляризации применяется для отбора признаков.

### Отбор признаков

Отбор признаков может быть глобальным или индивидуальным (instance-wise). Индивидуальный отбор признаков означает, что для каждого примера выбирается свое подмножество релевантных признаков. Такой подход ранее применялся для интерпретации обученных моделей ([Chen et al., 2018]($Learning to Explain: An Information-Theoretic Perspective on Model Interpretation$)).

Принцип работы TabNet во многом основан на индивидуальном отборе признаков. Решающие деревья обучаются выбирать наиболее статистически значимые признаки на каждом шаге, и архитектура TabNet следует той же идее.